## Natural language inference

### Contradictory dear watson
***

Project: https://www.kaggle.com/competitions/contradictory-my-dear-watson

### Starter guide

https://keras.io/guides/keras_nlp/getting_started/

#### Importar liberias

In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split

print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)

E0000 00:00:1727219194.170159    7381 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D0924 23:06:34.179037531    7381 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0924 23:06:34.179054173    7381 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0924 23:06:34.179057512    7381 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0924 23:06:34.179059953    7381 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0924 23:06:34.179062423    7381 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

TensorFlow version: 2.16.1
KerasNLP version: 0.14.4


#### Conf

In [2]:
try:
    # detect and init the TPU
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
except ValueError:
    strategy = tf.distribute.get_strategy()  # default strategy if no TPU available

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1727219200.467701    7381 service.cc:145] XLA service 0x576641154ca0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727219200.467776    7381 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1727219200.467782    7381 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1727219200.467785    7381 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1727219200.467788    7381 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1727219200.467791    7381 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1727219200.467794    7381 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1727219200.467797    7381 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1727219200.467800    7381 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [3]:
RESULT_DICT = {
    0 : "entailment",
    1 : "neutral",
    2 : "contradiction"
}

#### Data

In [4]:
df_train_data = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
df_submit_data = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')

In [5]:
df_train, df_test = train_test_split(df_train_data, test_size=0.2, random_state=42)

### Intermediate model

**Fine tuning a pretrained BERT backbone**

#### Data prep

In [7]:
# Asegúrate de que 'premise' y 'hypothesis' sean listas de strings
premises = df_train['premise'].astype(str).tolist()
hypothesis = df_train['hypothesis'].astype(str).tolist()


# Define las labels (etiquetas) para entrenar
labels = tf.convert_to_tensor(df_train['label'])

# Verifica que todas las entradas tengan la misma longitud
assert len(premises) == len(hypothesis) == len(labels), "Las dimensiones de las entradas no coinciden."

# Crea un dataset a partir de las entradas
train_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypothesis), labels)).batch(32)


#### Define model

In [ ]:
# Define el modelo dentro del scope de la estrategia
with strategy.scope():
    classifier = keras_nlp.models.BertClassifier.from_preset(
        "bert_base_multi",
        num_classes=3
    )
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

#### Train model

In [ ]:
classifier.fit(train_dataset, epochs=10)

### Advance model

**Fine tuning with user-controlled preprocessing**

#### Data prep

Separate preprocessing from the same preset

Each model architecture has a parallel preprocessor Layer with its own from_preset constructor. Using the same preset for this Layer will return the matching preprocessor as the task.

In [20]:
# Asegúrate de que 'premise' y 'hypothesis' sean listas de strings
premises = df_train['premise'].astype(str).tolist()
hypotheses = df_train['hypothesis'].astype(str).tolist()
labels = tf.convert_to_tensor(df_train['label'])

assert len(premises) == len(hypotheses) == len(labels), "Las dimensiones de las entradas no coinciden."

train_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypotheses), labels))

**Preprocesador Bert**

In [21]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_base_multi",
    sequence_length=512,
)

# Usa map() para aplicar el preprocesador
# Usa map() para aplicar el preprocesador
train_cached = (
    train_dataset
    .map(lambda x, y: (
        preprocessor(x[0]['token_ids']), 
        preprocessor(x[1]['token_ids']),
        y  # Agrega las etiquetas al conjunto de datos
    ), num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)


TypeError: in user code:

    File "/tmp/ipykernel_7381/1154158376.py", line 13, in None  *
        preprocessor(x[1]['token_ids']),

    TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got 'token_ids'


#### Define model

In [18]:
with strategy.scope():
    classifier = keras_nlp.models.BertClassifier.from_preset(
        "bert_base_multi", 
        num_classes=3  # Cambia esto según tus datos
    )
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

#### Train model

In [19]:
classifier.fit(
    train_cached,
    epochs=3,
)

ValueError: in user code:

    File "/usr/local/lib/python3.10/site-packages/keras_nlp/src/models/task.py", line 75, in preprocess_samples  *
        return self.preprocessor(x, y=y, sample_weight=sample_weight)
    File "/usr/local/lib/python3.10/site-packages/keras_nlp/src/layers/preprocessing/preprocessing_layer.py", line 48, in __call__  *
        outputs = super().__call__(*args, **kwargs)
    File "/usr/local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/site-packages/keras_nlp/src/models/bert/bert_preprocessor.py", line 155, in call
        x = convert_inputs_to_list_of_tensor_segments(x)
    File "/usr/local/lib/python3.10/site-packages/keras_nlp/src/utils/keras_utils.py", line 82, in convert_inputs_to_list_of_tensor_segments
        raise ValueError(

    ValueError: Exception encountered when calling BertPreprocessor.call().
    
    [1

In [22]:
# Asegúrate de que 'premise' y 'hypothesis' sean listas de strings
premises = df_train['premise'].astype(str).tolist()
hypotheses = df_train['hypothesis'].astype(str).tolist()
labels = tf.convert_to_tensor(df_train['label'])

assert len(premises) == len(hypotheses) == len(labels), "Las dimensiones de las entradas no coinciden."

train_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypotheses), labels))

preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_base_multi",
    sequence_length=512,
)

train_cached = (
    train_dataset
    .map(lambda x, y: (
        preprocessor(x[0]), 
        preprocessor(x[1]),
        y
    ), num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

with strategy.scope():
    classifier = keras_nlp.models.BertClassifier.from_preset(
        "bert_base_multi", 
        num_classes=3
    )
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )




In [23]:
classifier.fit(
    train_cached,
    epochs=3,
)

ValueError: in user code:

    File "/usr/local/lib/python3.10/site-packages/keras_nlp/src/models/task.py", line 75, in preprocess_samples  *
        return self.preprocessor(x, y=y, sample_weight=sample_weight)
    File "/usr/local/lib/python3.10/site-packages/keras_nlp/src/layers/preprocessing/preprocessing_layer.py", line 48, in __call__  *
        outputs = super().__call__(*args, **kwargs)
    File "/usr/local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/site-packages/keras_nlp/src/models/bert/bert_preprocessor.py", line 155, in call
        x = convert_inputs_to_list_of_tensor_segments(x)
    File "/usr/local/lib/python3.10/site-packages/keras_nlp/src/utils/keras_utils.py", line 82, in convert_inputs_to_list_of_tensor_segments
        raise ValueError(

    ValueError: Exception encountered when calling BertPreprocessor.call().
    
    [1